# Ray RLlib Multi-Armed Bandits - Market Bandit Example

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

Now that we've learned about multi-armed bandits and methods for optimizing rewards, let's look at real-world applications, starting with a stock market example. We'll also learn a little more about configuring RLlib trainers.

We'll load a dataset derived from this [NYU Stern table](http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html) that shows returns for nearly a century of market data, including dividends and adjustments for inflation. The `market.tsv` file in this folder contains the data.

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# Some properties we'll need:
DEFAULT_MAX_INFLATION = 100.0
DEFAULT_TICKERS = ["sp500", "t.bill", "t.bond", "corp"]
DEFAULT_DATA_FILE = os.path.abspath(os.path.curdir) + '/market.tsv'  # full path

In [4]:
def load_market_data (file_name):
    with open(file_name, "r") as f:
        return pd.read_table(f)

In [5]:
df = load_market_data(DEFAULT_DATA_FILE)
df

,year,inflation,sp500,t.bill,t.bond,corp
0,1928,-1.15,45.49,4.28,2.01,4.42
1,1929,0.00,-8.30,3.16,4.20,3.02
2,1930,-2.67,-23.07,7.42,7.41,3.30
3,1931,-8.93,-38.33,12.34,7.00,-7.41
4,1932,-10.30,1.85,12.68,21.28,37.78
...,...,...,...,...,...,...
87,2015,0.12,1.26,-0.07,1.16,-0.82
88,2016,1.26,10.38,-0.93,-0.56,8.99
89,2017,2.13,19.07,-1.17,0.66,7.44
90,2018,2.44,-6.51,-0.49,-2.40,-5.08


As you can see the data spans 92 years, from 1928 to 2019. The columns represent:
  * the year
  * inflation rate at the time
  * [S&P500](https://en.wikipedia.org/wiki/S%26P_500_Index) (composite stock index)
  * [Treasury Bills](https://www.investopedia.com/terms/t/treasurybill.asp) (short-term gov bonds)
  * [Treasury Bonds](https://www.investopedia.com/terms/t/treasurybond.asp) (long-term gov bonds)
  * [Moody's Baa Corporate Bonds](https://en.wikipedia.org/wiki/Moody%27s_Investors_Service#Moody's_credit_ratings) (moderate risk)

In [6]:
df.describe()

,year,inflation,sp500,t.bill,t.bond,corp
count,92.00000,92.000000,92.000000,92.000000,92.000000,92.000000
mean,1973.50000,3.041957,8.413261,0.434239,2.166413,4.216630
std,26.70206,3.803579,19.619605,3.573035,8.126432,8.625809
min,1928.00000,-10.300000,-38.900000,-12.050000,-14.570000,-14.850000
25%,1950.75000,1.415000,-2.740000,-1.185000,-2.620000,-1.322500
50%,1973.50000,2.750000,10.515000,0.590000,1.070000,3.910000
75%,1996.25000,4.275000,20.622500,2.117500,7.037500,9.287500
max,2019.00000,14.390000,58.200000,12.680000,25.140000,37.780000


"Corp" refers to corporate bonds.

## Analysis of the Data

What are the worst case and best case scenarios? In other words, if one could predict the future market performance, what are the possible ranges of total failure vs. total success over the past century? By "total", we mean what if you had all your money in a given year invested in the worst performing _sector_ (S&P500 or T bills or ...) or the best performing sector for that year.

In [7]:
n_years = len(df)
min_list = []
max_list = []

for i in range(n_years):
    row = df.iloc[i, 2:]
    min_list.append(min(row))
    max_list.append(max(row))
    
print("{:5.2f}% worst case annualized".format(sum(min_list) / n_years))
print("{:5.2f}% best case annualized".format(sum(max_list) / n_years))

-5.64% worst case annualized
15.18% best case annualized


In [8]:
from bokeh_util import plot_line, plot_line_with_stddev, plot_between_lines, plot_cumulative_regret

import bokeh
bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [9]:
min_max = pd.DataFrame.from_dict({'year': df['year'], 'min':min_list, 'max':max_list})
min_max

,year,min,max
0,1928,2.01,45.49
1,1929,-8.30,4.20
2,1930,-23.07,7.42
3,1931,-38.33,12.34
4,1932,1.85,37.78
...,...,...,...
87,2015,-0.82,1.26
88,2016,-0.93,10.38
89,2017,-1.17,19.07
90,2018,-6.51,-0.49


In [10]:
plot_between_lines(min_max, x_col='year', lower_col='min', upper_col='max', 
                   title='Best to Worst', x_axis_label='year', y_axis_label='%')

## Defining an Environment

Now let's define a Gym environment so that we can train a contextual bandit to optimize annual investments over that period.

In [11]:
import gym
from gym.spaces import Discrete, Box
from gym.utils import seeding
import numpy as np
import random

This is the bandit we'll use to represent the market "environment".

In [12]:
class MarketBandit (gym.Env):
    
    def __init__ (self, config={}):
        self.max_inflation = config.get('max-inflation', DEFAULT_MAX_INFLATION)
        self.tickers = config.get('tickers', DEFAULT_TICKERS)
        self.data_file = config.get('data-file', DEFAULT_DATA_FILE)
        print(f"MarketBandit: max_inflation: {self.max_inflation}, tickers: {self.tickers}, data file: {self.data_file} (config: {config})")

        self.action_space = Discrete(4)
        self.observation_space = Box(
            low  = -self.max_inflation,
            high =  self.max_inflation,
            shape=(1, )
        )
        self.df = load_market_data(self.data_file)
        self.cur_context = None


    def reset (self):
        self.year = self.df["year"].min()
        self.cur_context = self.df.loc[self.df["year"] == self.year]["inflation"][0]
        self.done = False
        self.info = {}

        return [self.cur_context]


    def step (self, action):
        if self.done:
            reward = 0.
            regret = 0.
        else:
            row = self.df.loc[self.df["year"] == self.year]

            # calculate reward
            ticker = self.tickers[action]
            reward = float(row[ticker])

            # calculate regret
            max_reward = max(map(lambda t: float(row[t]), self.tickers))
            regret = round(max_reward - reward)

            # update the context
            self.cur_context = float(row["inflation"])

            # increment the year
            self.year += 1

            if self.year >= self.df["year"].max():
                self.done = True

        context = [self.cur_context]
        #context = self.observation_space.sample()

        self.info = {
            "regret": regret,
            "year": self.year
        }
         
        return [context, reward, self.done, self.info]


    def seed (self, seed=None):
        """Sets the seed for this env's random number generator(s).
        Note:
            Some environments use multiple pseudorandom number generators.
            We want to capture all such seeds used in order to ensure that
            there aren't accidental correlations between multiple generators.
        Returns:
            list<bigint>: Returns the list of seeds used in this env's random
              number generators. The first value in the list should be the
              "main" seed, or the value which a reproducer should pass to
              'seed'. Often, the main seed equals the provided 'seed', but
              this won't be true if seed=None, for example.
        """
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

Let's see it in action:

In [13]:
bandit = MarketBandit()
bandit.reset()

for i in range(10):
    action = bandit.action_space.sample()
    obs = bandit.step(action)
    print(action, obs)

MarketBandit: max_inflation: 100.0, tickers: ['sp500', 't.bill', 't.bond', 'corp'], data file: /Users/deanwampler/projects/anyscale/academy/academy-git/ray-rllib/multi-armed-bandits/market.tsv (config: {})
3 [[-1.15], 4.42, False, {'regret': 41, 'year': 1929}]
3 [[0.0], 3.02, False, {'regret': 1, 'year': 1930}]
2 [[-2.67], 7.41, False, {'regret': 0, 'year': 1931}]
2 [[-8.93], 7.0, False, {'regret': 5, 'year': 1932}]
3 [[-10.3], 37.78, False, {'regret': 0, 'year': 1933}]
3 [[-5.19], 19.15, False, {'regret': 39, 'year': 1934}]
0 [[3.48], -4.51, False, {'regret': 19, 'year': 1935}]
0 [[2.55], 43.09, False, {'regret': 0, 'year': 1936}]
2 [[1.03], 3.94, False, {'regret': 27, 'year': 1937}]
2 [[3.73], -2.26, False, {'regret': 0, 'year': 1938}]


/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


We can use this environment in a kind of *monte carlo simulation* to measure a baseline for what the rewards would be over a long period if you always used a random action.

In [14]:
done = 1
reward_list = []
iterations = 10000 #50000

for i in range(iterations):
    if done == 1:
        bandit.reset()

    action = bandit.action_space.sample()
    obs = bandit.step(action)
    context, reward, done, info = obs
    reward_list.append(reward)
    #print(action, context, reward, done, info)

In [15]:
df_mc = pd.DataFrame(reward_list, columns=["reward"])
df_mc.mean()

reward    3.86364
dtype: float64

Depending on the number of iterations, you'll probably get a value approaching 3.75% as a baseline for random actions. That's more than the -5.64% worst case and must less than 15.18% best case for the reward!

In [16]:
from bokeh_util import plot_line, plot_line_with_stddev, plot_cumulative_regret

In [17]:
plot_line(df_mc, x_col='index', y_col='reward', title='Reward Over Time')

([image](../../images/rllib/MarketReward-Random.png))

Yes, it looks quite random...

## Training a policy in RLlib

Now let's train a policy using our contextual bandit, specifically using _Linear Thompson Sampling_ in RLlib. Hopefully it will do better than the random results we just computed!

Recall in the `__init__()` method for `MarketBandit` that we set some parameters from the passed in `config` object (with defaults). We don't construct this explicitly ourselves. Rather, RLlib will do this. So, we need to construct the canonical `config` object we want to use. To do this, we use the idioms shown in the next several cells:

In [18]:
from ray.rllib.agents.trainer import with_base_config, with_common_config
from ray.rllib.contrib.bandits.agents.lin_ts import TS_CONFIG
from ray.rllib.contrib.bandits.agents.lin_ts import LinTSTrainer
from ray import tune

We need a custom config object with our parameters for `MarketBandit`. We do this building on the default `TS_CONFIG` object for _LinTS_:

In [19]:
market_config = with_base_config(TS_CONFIG, {
    "env":           MarketBandit,
    'max-inflation': DEFAULT_MAX_INFLATION,
    'tickers':       DEFAULT_TICKERS,
    'data-file':     DEFAULT_DATA_FILE
})

stop = {
    "training_iteration": 100
}

Also, we'll define a custom trainer, which builds on the `LinTSTrainer`, with "updates". Note that it's the first argument we'll pass to `tune.run()` in the following cell. When all we need is `LinTSTrainer`, as is, and no extra custom config settings, we can just pass the string `contrib/LinTS` to `tune.run()`.  

In [20]:
MarketLinTSTrainer = LinTSTrainer.with_updates(
    name="MarketLinTSTrainer",
    default_config=market_config,      # Will be merged with Trainer.COMMON_CONFIG (rllib/agent/trainer.py)
    #default_policy=[somePolicyClass]  # If we had a policy...
)

In [21]:
analysis = tune.run(
    MarketLinTSTrainer,
    config=market_config,
    stop=stop,
    num_samples=3,    
    checkpoint_at_end=True,
    verbose=2            # Change to 0 or 1 to reduce the output.
)

2020-06-12 06:49:49,149	INFO resource_spec.py:212 -- Starting Ray with 3.71 GiB memory available for workers and up to 1.87 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-12 06:49:49,605	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


Trial name,status,loc
MarketLinTSTrainer_MarketBandit_00000,RUNNING,
MarketLinTSTrainer_MarketBandit_00001,PENDING,
MarketLinTSTrainer_MarketBandit_00002,PENDING,


(pid=32065) 2020-06-12 06:49:54,723	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=32065) 2020-06-12 06:49:54,727	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=32065) /Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=32065)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=32061) 2020-06-12 06:49:54,723	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=32061) 2020-06-12 06:49:54,727	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=32061) /Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: W

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,192.168.1.149:32065,3,0.652767,300,614.653
MarketLinTSTrainer_MarketBandit_00001,RUNNING,192.168.1.149:32061,2,0.44261,200,632.755
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:32067,2,0.442028,200,628.735


Result for MarketLinTSTrainer_MarketBandit_00002:
  custom_metrics: {}
  date: 2020-06-12_06-50-00
  done: false
  episode_len_mean: 91.0
  episode_reward_max: 690.84
  episode_reward_mean: 578.0900000000001
  episode_reward_min: 409.53000000000003
  episodes_this_iter: 1
  episodes_total: 20
  experiment_id: 6b0fea5919674b13aaff472133a30772
  experiment_tag: '2'
  grad_time_ms: 0.375
  hostname: DWAnyscaleMBP.local
  info:
    grad_time_ms: 0.375
    learner:
      cumulative_regret: 16518.0
      update_latency: 0.0002009868621826172
    num_steps_sampled: 1900
    num_steps_trained: 1900
    opt_peak_throughput: 2665.589
    opt_samples: 1.0
    sample_peak_throughput: 534.027
    sample_time_ms: 1.873
    update_time_ms: 0.002
  iterations_since_restore: 19
  learner:
    cumulative_regret: 16518.0
    update_latency: 0.0002009868621826172
  node_ip: 192.168.1.149
  num_healthy_workers: 0
  num_steps_sampled: 1900
  num_steps_trained: 1900
  off_policy_estimator: {}
  opt_peak_thro

...

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,192.168.1.149:32065,77,20.0452,7700,607.18
MarketLinTSTrainer_MarketBandit_00001,RUNNING,192.168.1.149:32061,77,20.1274,7700,623.714
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:32067,76,19.914,7600,602.757


Result for MarketLinTSTrainer_MarketBandit_00002:
  custom_metrics: {}
  date: 2020-06-12_06-50-15
  done: false
  episode_len_mean: 91.0
  episode_reward_max: 753.3299999999998
  episode_reward_mean: 602.252380952381
  episode_reward_min: 393.92000000000024
  episodes_this_iter: 1
  episodes_total: 84
  experiment_id: 6b0fea5919674b13aaff472133a30772
  experiment_tag: '2'
  grad_time_ms: 0.445
  hostname: DWAnyscaleMBP.local
  info:
    grad_time_ms: 0.445
    learner:
      cumulative_regret: 64810.0
      update_latency: 0.0002727508544921875
    num_steps_sampled: 7700
    num_steps_trained: 7700
    opt_peak_throughput: 2247.149
    opt_samples: 1.0
    sample_peak_throughput: 518.263
    sample_time_ms: 1.93
    update_time_ms: 0.002
  iterations_since_restore: 77
  learner:
    cumulative_regret: 64810.0
    update_latency: 0.0002727508544921875
  node_ip: 192.168.1.149
  num_healthy_workers: 0
  num_steps_sampled: 7700
  num_steps_trained: 7700
  off_policy_estimator: {}
  opt_

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,192.168.1.149:32065,96,25.0062,9600,607.052
MarketLinTSTrainer_MarketBandit_00001,RUNNING,192.168.1.149:32061,96,25.1304,9600,615.849
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:32067,95,24.8833,9500,602.741


Result for MarketLinTSTrainer_MarketBandit_00002:
  custom_metrics: {}
  date: 2020-06-12_06-50-20
  done: false
  episode_len_mean: 91.0
  episode_reward_max: 753.3299999999998
  episode_reward_mean: 602.1188999999999
  episode_reward_min: 393.92000000000024
  episodes_this_iter: 1
  episodes_total: 105
  experiment_id: 6b0fea5919674b13aaff472133a30772
  experiment_tag: '2'
  grad_time_ms: 0.494
  hostname: DWAnyscaleMBP.local
  info:
    grad_time_ms: 0.494
    learner:
      cumulative_regret: 80876.0
      update_latency: 0.0002799034118652344
    num_steps_sampled: 9600
    num_steps_trained: 9600
    opt_peak_throughput: 2023.399
    opt_samples: 1.0
    sample_peak_throughput: 530.086
    sample_time_ms: 1.886
    update_time_ms: 0.002
  iterations_since_restore: 96
  learner:
    cumulative_regret: 80876.0
    update_latency: 0.0002799034118652344
  node_ip: 192.168.1.149
  num_healthy_workers: 0
  num_steps_sampled: 9600
  num_steps_trained: 9600
  off_policy_estimator: {}
  o

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,TERMINATED,,100,26.1341,10000,603.31
MarketLinTSTrainer_MarketBandit_00001,TERMINATED,,100,26.2512,10000,615.852
MarketLinTSTrainer_MarketBandit_00002,TERMINATED,,100,26.2863,10000,604.052


## Analyzing the results

Let's analyze the rewards and cumulative regrets of these trials.

In [22]:
df_ts = pd.DataFrame()

for key, df_trial in analysis.trial_dataframes.items():
    df_ts = df_ts.append(df_trial, ignore_index=True)
    
df_ts.head()

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_steps_trained,num_steps_sampled,sample_time_ms,grad_time_ms,update_time_ms,...,info/update_time_ms,info/opt_peak_throughput,info/sample_peak_throughput,info/opt_samples,learner/cumulative_regret,learner/update_latency,perf/cpu_util_percent,perf/ram_util_percent,info/learner/cumulative_regret,info/learner/update_latency
0,566.57,566.57,566.570000,91.0,1,100,100,1.938,0.356,0.001,...,0.001,2811.761,515.981,1.0,950.0,0.000200,59.8,67.0,950.0,0.000200
1,593.63,566.57,580.100000,91.0,1,200,200,1.962,0.361,0.001,...,0.001,2767.239,509.785,1.0,1780.0,0.000185,NaN,NaN,1780.0,0.000185
2,683.76,566.57,614.653333,91.0,1,300,300,1.722,0.336,0.001,...,0.001,2979.544,580.808,1.0,2655.0,0.000256,NaN,NaN,2655.0,0.000256
3,683.76,518.81,590.692500,91.0,1,400,400,1.766,0.384,0.001,...,0.001,2601.764,566.300,1.0,3586.0,0.000180,44.0,67.0,3586.0,0.000180
4,683.76,458.89,564.332000,91.0,1,500,500,1.686,0.363,0.001,...,0.001,2755.241,593.153,1.0,4416.0,0.000342,NaN,NaN,4416.0,0.000342


In [23]:
rewards = df_ts \
    .groupby("num_steps_trained")["episode_reward_mean"] \
    .aggregate(["mean", "max", "min", "std"])

rewards

,mean,max,min,std
num_steps_trained,,,,
100,590.423333,674.830000,529.870000,75.366349
200,613.863333,632.755000,580.100000,29.308908
300,638.316667,669.523333,614.653333,28.202050
400,630.655833,693.565000,590.692500,55.141375
500,621.588667,688.688000,564.332000,62.759559
...,...,...,...,...
9600,608.339833,615.848700,602.118900,6.954921
9700,607.929500,616.127800,601.669900,7.421364
9800,609.060533,617.620400,602.411200,7.782515


In [24]:
plot_line_with_stddev(rewards, x_col='num_steps_trained', y_col='mean', stddev_col='std', 
                      title='Rewards vs. Steps', x_axis_label='step', y_axis_label='reward')

The rewards reach what appears to be nearly optimzal by 3000 steps, then shows some slow improvement above 8000.

In [25]:
regrets = df_ts \
    .groupby("num_steps_trained")["learner/cumulative_regret"] \
    .aggregate(["mean", "max", "min", "std"])

regrets

,mean,max,min,std
num_steps_trained,,,,
100,916.000000,1000.0,798.0,105.204563
200,1699.333333,1780.0,1632.0,74.895483
300,2484.333333,2655.0,2319.0,168.063480
400,3305.333333,3586.0,2978.0,306.674638
500,4136.333333,4416.0,3778.0,326.193705
...,...,...,...,...
9600,80126.333333,80876.0,79092.0,925.440616
9700,80938.333333,81709.0,79827.0,986.164963
9800,81758.000000,82479.0,80666.0,961.755166


In [26]:
plot_cumulative_regret(regrets)

## Evaluating the Trained Policy

Overall, how well did the trained policy perform? The results should be better than random, but less than the best case.

In [27]:
print("{:5.2f}% optimized return annualized".format(max(rewards["mean"]) / n_years))

 6.94% optimized return annualized


That's better than the random action baseline of 3.75%, but no where near the best case scenario of 15.18% return. Hence, our regrets grow...

Note that investing solely in the S&P stock index which would have produced better than 8% return over that period -- that is, if one could wait 92 years. However, investing one's entire portfolio into stocks can become quite a risky policy in the short-term, so we were exploring how to balance a portfolio given only limited information.

In any case, the contextual bandit performed well considering that it could only use *inflation* for the context of its decisions, and could only take actions once each year.

## Exercise 1

Try using a `LinUCBTrainer`-based trainer. How does the annualized return compare?

---

## Extra - Restoring from a Checkpoint

In the previous lesson, [05 Thompson Sampling](05-Thompson-Sampling.ipynb), we showed how to restore a trainer from a checkpoint, but almost "in passing". Let's use this feature again, this time with our custom trainer class `MarketLinTSTrainer`.

In [28]:
trial = analysis.trials[0]
path = trial.checkpoint.value
print(f'checkpoint_path: {path}')

checkpoint_path: /Users/deanwampler/ray_results/MarketLinTSTrainer/MarketLinTSTrainer_MarketBandit_0_2020-06-12_06-49-50xz04tekh/checkpoint_100/checkpoint-100


In [31]:
trainer = MarketLinTSTrainer(market_config)  # create instance and then restore from checkpoint
trainer.restore(path)

2020-06-12 06:57:17,040	INFO trainable.py:217 -- Getting current IP.
2020-06-12 06:57:17,041	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-06-12 06:57:17,044	INFO trainable.py:217 -- Getting current IP.
2020-06-12 06:57:17,044	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: /Users/deanwampler/ray_results/MarketLinTSTrainer/MarketLinTSTrainer_MarketBandit_0_2020-06-12_06-49-50xz04tekh/checkpoint_100/checkpoint-100
2020-06-12 06:57:17,045	INFO trainable.py:430 -- Current state after restoring: {'_iteration': 100, '_timesteps_total': 10000, '_time_total': 26.134074449539185, '_episodes_total': 109}


MarketBandit: max_inflation: 100.0, tickers: ['sp500', 't.bill', 't.bond', 'corp'], data file: /Users/deanwampler/projects/anyscale/academy/academy-git/ray-rllib/multi-armed-bandits/market.tsv (config: {})


Get model to plot arm weights distribution

In [32]:
model = trainer.get_policy().model
means = [model.arms[i].theta.numpy() for i in range(3)]
covs = [model.arms[i].covariance.numpy() for i in range(3)]
means, covs, model.arms[0].theta.numpy()

([array([1.3471998], dtype=float32),
  array([-0.29587987], dtype=float32),
  array([-1.2319467], dtype=float32)],
 [array([[6.7857027e-06]], dtype=float32),
  array([[4.2226464e-05]], dtype=float32),
  array([[5.0314273e-05]], dtype=float32)],
 array([1.3471998], dtype=float32))

A final note; when you checkpoint the model, it will change how the training performs in this notebook, if you rerun the training! So, when doing experiments, be sure you are starting from scratch when that is desired!